In [1]:
import os

In [2]:
%pwd

'c:\\Users\\artit\\IKP_2025\\fraud_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\artit\\IKP_2025\\fraud_prediction'

In [5]:
import tensorflow as tf

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/ArtitayaN/fraud_prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="ArtitayaN"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d10f05c3bf59a9f946e535cddb121b4d48d5e8b9"

In [7]:
#$env:MLFLOW_TRACKING_URI=https://https://dagshub.com/ArtitayaN/Fraud-Detection-MLOps.mlflow

#$env:MLFLOW_TRACKING_USERNAME=ArtitayaN

#$env:MLFLOW_TRACKING_PASSWORD=d10f05c3bf59a9f946e535cddb121b4d48d5e8b9


In [8]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_num_features: list
    params_batch_size: int

In [10]:
from fraud_prediction.constants import *
from fraud_prediction.utils.common import read_yaml, create_directories, save_json

In [39]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/fraud_0.1origbase.csv",
            mlflow_uri="https://dagshub.com/ArtitayaN/fraud_prediction.mlflow",
            all_params=self.params,
            params_num_features=self.params.NUM_FEATURES,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [40]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from fraud_prediction.utils.common import save_json 
import os
import glob
from sklearn.preprocessing import MinMaxScaler
from mlflow.models.signature import infer_signature

In [36]:
import os
import glob
data_dir = "artifacts/data_ingestion/" # ลองใส่ path ตามใน config ของคุณ
files = glob.glob(os.path.join(data_dir, "**/*.csv"), recursive=True)
print(f"พบไฟล์ CSV ทั้งหมด: {files}")

พบไฟล์ CSV ทั้งหมด: ['artifacts/data_ingestion\\fraud_0.1origbase.csv']


In [41]:
class Evaluation:
    def __init__(self, config: EvaluationConfig): 
        self.config = config
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(str(path))
    
    def _prepare_validation_data(self):
        """โหลดและเตรียมข้อมูลสำหรับวัดผล (ลอจิกเดียวกับตอนเทรน)"""
        data_dir = self.config.training_data
        print(f"DEBUG: กำลังค้นหาไฟล์ใน Path -> {os.path.abspath(data_dir)}")

        if data_dir.endswith('csv') and os.path.isfile(data_dir):
            target_file = data_dir
        
        else:
            csv_files = glob.glob(os.path.join(data_dir, "*.csv"))

            if not csv_files:
                csv_files = glob.glob(os.path.join(data_dir, "**", "*.csv"), recursive=True)

            if not csv_files:
                raise FileNotFoundError(f"Not found file .csv in: {os.path.abspath(data_dir)}")
        
            target_file = csv_files[0]
        
        print(f" Found data file: {target_file}")
        df = pd.read_csv(target_file)
        
        # 1. Feature Engineering (ต้องเหมือนตอนเทรน)
        df['diff_new_old_balance'] = df['newbalanceOrig'] - df['oldbalanceOrg']
        df['diff_new_old_destiny'] = df['newbalanceDest'] - df['oldbalanceDest']
        
        cols_to_drop = ['nameOrig', 'nameDest', 'isFlaggedFraud','step_weeks', 'step_days'] 
        df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

        # Important to put dtype = int bc about error while doing One-Hot
        df = pd.get_dummies(df, columns=['type'], dtype=int)
        df = df.dropna()

        # 2. แยก X, y
        X = df.drop(columns=['isFraud'])
        y = df['isFraud']

        # 3. Scaling (ในระบบจริงควรโหลด scaler ที่ Save ไว้มาใช้ แต่ตอนนี้ทำใหม่เพื่อ Test)
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X)
        
        self.X_valid = np.asarray(X_scaled).astype('float32')
        self.y_valid = np.asarray(y).astype('float32')

    def evaluation(self):
        # 1. Load model 
        self.model = self.load_model(self.config.path_of_model)
        self._prepare_validation_data()

        # Measure results by using Data Array
        self.score = self.model.evaluate(self.X_valid, self.y_valid)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        """ส่งผลลัพธ์ขึ้น DagsHub/MLflow """
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            # บันทึกโมเดลขึ้น Cloud (Model Registry)
            if tracking_url_type_store != "file":
                signature = infer_signature(self.X_valid, self.model.predict(self.X_valid))
                mlflow.keras.log_model(self.model, name = "FraudANNModel_V1" )#, signature=signature) , registered_model_name="FraudModel")
            else:
                mlflow.keras.log_model(self.model, name_= "model" )#,signature=signature) #, registered_model_name="FraudModel")

In [42]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
    print(f" Push data to DagsHub already!")
except Exception as e:
   raise e

DEBUG: กำลังค้นหาไฟล์ใน Path -> c:\Users\artit\IKP_2025\fraud_prediction\artifacts\data_ingestion\fraud_0.1origbase.csv
 Found data file: artifacts/data_ingestion/fraud_0.1origbase.csv
19884/19884 ━━━━━━━━━━━━━━━━━━━━ 16s 799us/step - accuracy: 0.9988 - loss: 0.0250
19884/19884 ━━━━━━━━━━━━━━━━━━━━ 10s 482us/step


2026/01/26 18:56:16 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


🏃 View run charming-crow-933 at: https://dagshub.com/ArtitayaN/fraud_prediction.mlflow/#/experiments/0/runs/785729c4f5e94acaa69651412eb7c1a8
🧪 View experiment at: https://dagshub.com/ArtitayaN/fraud_prediction.mlflow/#/experiments/0
 Push data to DagsHub already!
